# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [35]:
song_files = get_files('data/song_data')

In [36]:
filepath = song_files[0]

In [40]:
df = pd.read_json(filepath, lines=True)
df = df.where(pd.notnull(df), None)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARGSAFR1269FB35070,None,None,,Blingtones,SOTCKKY12AB018A141,Sonnerie lalaleulé hi houuu,29.54404,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [29]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values.tolist()[0]
song_data

['SOTCKKY12AB018A141',
 'Sonnerie lalaleulé hi houuu',
 'ARGSAFR1269FB35070',
 0,
 29.54404]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [35]:
song_table_insert = ("""
INSERT INTO songs (song_id, title, artist_id, year, duration)
    VALUES (%s, %s, %s, %s, %s);
""")
cur.execute(song_table_insert, song_data)
conn.commit()

UniqueViolation: duplicate key value violates unique constraint "songs_pkey"
DETAIL:  Key (song_id)=(SOTCKKY12AB018A141) already exists.


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [42]:
artist_data = df.where(pd.notnull(df), None)
artist_data = artist_data[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values.tolist()[0]
artist_data

['ARGSAFR1269FB35070', 'Blingtones', '', None, None]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [44]:
conn.rollback()
artist_table_insert = ("""
INSERT INTO artists (artist_id, name, location, latitude, longitude)
    VALUES (%s, %s, %s, %s, %s)
    ON CONFLICT ON CONSTRAINT artists_pkey 
      DO UPDATE SET
        (name, location, latitude, longitude) = (EXCLUDED.name, EXCLUDED.location, EXCLUDED.latitude, EXCLUDED.longitude);
""")
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [47]:
log_files = get_files('data/log_data')

In [48]:
filepath = log_files[0]
filepath

'/home/camilortte/Documents/Udacity/data-engineer-nanodegree/1-project/sparkifyPostgresDataModeling/data/log_data/2018/11/2018-11-03-events.json'

In [49]:
df = pd.read_json(filepath, lines=True)
df = df.where(pd.notnull(df), None)
df.rename(columns={'userId' :'user_id', 'firstName': 'first_name', 'lastName': 'last_name'}, inplace=True)
df.head()

,artist,auth,first_name,gender,itemInSession,last_name,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,user_id
0,None,Logged Out,None,None,0,None,None,free,None,PUT,Login,None,52,None,307,1541207073796,None,
1,None,Logged In,Celeste,F,1,Williams,None,free,"Klamath Falls, OR",GET,Home,1541077528796.0,52,None,200,1541207123796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
2,Mynt,Logged In,Celeste,F,2,Williams,166.94812,free,"Klamath Falls, OR",PUT,NextSong,1541077528796.0,52,Playa Haters,200,1541207150796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
3,Taylor Swift,Logged In,Celeste,F,3,Williams,230.47791,free,"Klamath Falls, OR",PUT,NextSong,1541077528796.0,52,You Belong With Me,200,1541207316796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
4,Amy Winehouse,Logged In,Celeste,F,4,Williams,229.85098,free,"Klamath Falls, OR",PUT,NextSong,1541077528796.0,52,Valerie,200,1541207546796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe


In [50]:
df = df[ (df.page == "NextSong") & (df.user_id.notnull())]
df = df.drop_duplicates(subset=['user_id'])
df.head()

,artist,auth,first_name,gender,itemInSession,last_name,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,user_id
2,Mynt,Logged In,Celeste,F,2,Williams,166.94812,free,"Klamath Falls, OR",PUT,NextSong,1541077528796.0,52,Playa Haters,200,1541207150796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
7,Maldita Nerea,Logged In,Anabelle,F,0,Simpson,241.162,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1541044398796.0,158,Supelicula,200,1541254670796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
8,Fluke,Logged In,Connar,M,0,Moreno,478.92853,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1540823606796.0,168,Bermuda,200,1541257880796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10)...",62
9,Habib KoitÃÂ©,Logged In,Jayden,M,0,Fox,285.1522,free,"New Orleans-Metairie, LA",PUT,NextSong,1541033612796.0,185,Din Din Wo,200,1541259368796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",101
10,The Kooks,Logged In,Sara,F,0,Johnson,132.25751,paid,"Winston-Salem, NC",PUT,NextSong,1540809153796.0,152,Eddie's Gun,200,1541260356796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95


In [51]:
t = pd.to_datetime(df['ts'], unit='ms')
df['ts'] = pd.to_datetime(df['ts'], unit='ms')
df.head()

,artist,auth,first_name,gender,itemInSession,last_name,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,user_id
2,Mynt,Logged In,Celeste,F,2,Williams,166.94812,free,"Klamath Falls, OR",PUT,NextSong,1541077528796.0,52,Playa Haters,200,2018-11-03 01:05:50.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
7,Maldita Nerea,Logged In,Anabelle,F,0,Simpson,241.162,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1541044398796.0,158,Supelicula,200,2018-11-03 14:17:50.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
8,Fluke,Logged In,Connar,M,0,Moreno,478.92853,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1540823606796.0,168,Bermuda,200,2018-11-03 15:11:20.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10)...",62
9,Habib KoitÃÂ©,Logged In,Jayden,M,0,Fox,285.1522,free,"New Orleans-Metairie, LA",PUT,NextSong,1541033612796.0,185,Din Din Wo,200,2018-11-03 15:36:08.796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",101
10,The Kooks,Logged In,Sara,F,0,Johnson,132.25751,paid,"Winston-Salem, NC",PUT,NextSong,1540809153796.0,152,Eddie's Gun,200,2018-11-03 15:52:36.796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95


In [52]:
time_data = {
    'timestamp': df['ts'].values,
    'hour': df['ts'].dt.hour.values,
    'day': df['ts'].dt.day.values, 
    'week_of_year': df['ts'].dt.isocalendar().week.values,  
    'month': df['ts'].dt.month.values,
    'year': df['ts'].dt.year.values,  
    'weekday': df['ts'].dt.weekday .values
}
column_labels = ['timestamp', 'hour', 'day', 'week_of_year', 'month', 'year', 'weekday']    

In [53]:
time_df = pd.DataFrame(time_data)
time_df.head()

,timestamp,hour,day,week_of_year,month,year,weekday
0,2018-11-03 01:05:50.796,1,3,44,11,2018,5
1,2018-11-03 14:17:50.796,14,3,44,11,2018,5
2,2018-11-03 15:11:20.796,15,3,44,11,2018,5
3,2018-11-03 15:36:08.796,15,3,44,11,2018,5
4,2018-11-03 15:52:36.796,15,3,44,11,2018,5


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [57]:
conn.rollback()
time_table_insert = ("""
INSERT INTO time (start_time, hour, day, week, month, year, weekday)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT ON CONSTRAINT time_pkey 
      DO NOTHING;
""")
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [58]:
user_df = df[ ['user_id', 'first_name', 'last_name', 'gender','level'] ]
user_df = user_df[ user_df['user_id'].notnull()]
user_df = user_df[user_df.user_id != '']
user_df

,user_id,first_name,last_name,gender,level
2,53,Celeste,Williams,F,free
7,69,Anabelle,Simpson,F,free
8,62,Connar,Moreno,M,free
9,101,Jayden,Fox,M,free
10,95,Sara,Johnson,F,paid
27,10,Sylvie,Cruz,F,free
29,15,Lily,Koch,F,paid
104,63,Ayla,Johnson,F,free
106,49,Chloe,Cuevas,F,free
109,6,Cecilia,Owens,F,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [59]:
user_table_insert = ("""
INSERT INTO users (user_id, first_name, last_name, gender, level)
    VALUES (%s, %s, %s, %s, %s)
    ON CONFLICT ON CONSTRAINT users_pkey 
      DO UPDATE SET
        (first_name, last_name, gender, level) = (EXCLUDED.first_name, EXCLUDED.last_name, EXCLUDED.gender, EXCLUDED.level);
""")
for i, row in user_df.iterrows():
    #print(row.values)
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [70]:
# conn.rollback()
songplay_table_insert = ("""
INSERT INTO songplays (
  start_time,
  user_id,
  level,
  song_id,
  artist_id,
  session_id,
  location,
  usert_agent
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
""")
song_select = ("""
SELECT songs.song_id, songs.artist_id
FROM songs
INNER JOIN artists ON songs.artist_id = artists.artist_id
WHERE title = %s AND artists.name = %s AND duration = %s
""")
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
    
    # insert songplay record
    if songid and artistid:
        songplay_data = (row.ts, row.user_id, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
        cur.execute(songplay_table_insert, songplay_data)
        conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.